In [1]:
import requests
import pandas as pd
import random

In [2]:
user = 'thatchell.aca@gmail.com'
password = 'Bfpn8T5zgcmwr-*'

In [3]:
session = requests.Session()
login_url = 'https://www.space-track.org/ajaxauth/login'
login_payload = {'identity': user, 'password': password}
login_resp = session.post(login_url, data=login_payload)
login_resp.raise_for_status()

In [4]:
# Query SATCAT for all objects with perigee <= 2000 km
meta_url = (
    'https://www.space-track.org/basicspacedata/query/class/satcat/'
    'PERIGEE/0--3000/' # filters perigee between 0 and 2000 km
    'orderby/NORAD_CAT_ID asc/' # sory by ID ascending
    'format/json'
)
meta_resp = session.get(meta_url)
meta_resp.raise_for_status()
sat_meta = meta_resp.json()

In [5]:
# Extract list of NORAD IDs
leo_ids = [entry['NORAD_CAT_ID'] for entry in sat_meta]


In [6]:
sample_ids = random.sample(leo_ids, 100)
print(f'Sampling {len(sample_ids)} LEO satellites...')

Sampling 100 LEO satellites...


In [7]:
# Define 2-year window
start_date = '2019-01-01'
end_date = '2021-01-01'

In [11]:
sample_ids_int = []
for norad in sample_ids:
    try:
        sample_ids_int.append(int(norad))
    except ValueError:
        continue

filtered_ids = []
for norad in sample_ids_int:
    if norad >= 30000 and norad <= 50000:
        filtered_ids.append(norad)

all_records = []
for norad in filtered_ids:
    tle_url = (
        'https://www.space-track.org/basicspacedata/query/class/gp_history/'
        f'NORAD_CAT_ID/{norad}/'
        f'CREATION_DATE/{start_date}--{end_date}/'
        'orderby/CREATION_DATE%20asc/'
        'format/json'
    )
    resp = session.get(tle_url)
    resp.raise_for_status()
    history = resp.json()

    if not history or 'ERROR' in history[0]:
        print(f'Skipping NORAD {norad}: no history in {start_date}--{end_date}')
        continue

    print(f'NORAD {norad} record keys:', list(history[0].keys()))
    
    df = pd.DataFrame(history)
    df.columns = df.columns.str.strip().str.upper()

    ts_col = None
    for candidate in ('CREATION_DATE', 'EPOCH', 'REF_TIME', 'UPDATE_DATE'):
        if candidate in df.columns:
            ts_col = candidate
            break
    if ts_col is None:
        print(f'No timestamp column found for NORAD {norad}, columns are {df.columns.tolist()}')
        continue

    df['EPOCH'] = pd.to_datetime(df[ts_col])

    df['DATE'] = df['EPOCH'].dt.date

    keep = ['DATE', 'NORAD_CAT_ID', 'SEMIMAJOR_AXIS', 'APOAPSIS', 'PERIAPSIS', 'MEAN_MOTION', 'ECCENTRICITY', 'INCLINATION', 'ARG_OF_PERICENTER']
    if 'NORAD_CAT_ID' not in df.columns:
        df['NORAD_CAT_ID'] = norad

    df = df[[c for c in keep if c in df.columns]]
    daily = df.groupby('DATE', as_index=False).first()
    daily['NORAD_CAT_ID'] = norad
    all_records.append(daily)
    print(f'Collected {norad} ({len(all_records)}/{len(sample_ids)})')

Skipping NORAD 39557: no history in 2019-01-01--2021-01-01
Skipping NORAD 38847: no history in 2019-01-01--2021-01-01
NORAD 39359 record keys: ['CCSDS_OMM_VERS', 'COMMENT', 'CREATION_DATE', 'ORIGINATOR', 'OBJECT_NAME', 'OBJECT_ID', 'CENTER_NAME', 'REF_FRAME', 'TIME_SYSTEM', 'MEAN_ELEMENT_THEORY', 'EPOCH', 'MEAN_MOTION', 'ECCENTRICITY', 'INCLINATION', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'EPHEMERIS_TYPE', 'CLASSIFICATION_TYPE', 'NORAD_CAT_ID', 'ELEMENT_SET_NO', 'REV_AT_EPOCH', 'BSTAR', 'MEAN_MOTION_DOT', 'MEAN_MOTION_DDOT', 'SEMIMAJOR_AXIS', 'PERIOD', 'APOAPSIS', 'PERIAPSIS', 'OBJECT_TYPE', 'RCS_SIZE', 'COUNTRY_CODE', 'LAUNCH_DATE', 'SITE', 'DECAY_DATE', 'FILE', 'GP_ID', 'TLE_LINE0', 'TLE_LINE1', 'TLE_LINE2']
Collected 39359 (1/100)
NORAD 45264 record keys: ['CCSDS_OMM_VERS', 'COMMENT', 'CREATION_DATE', 'ORIGINATOR', 'OBJECT_NAME', 'OBJECT_ID', 'CENTER_NAME', 'REF_FRAME', 'TIME_SYSTEM', 'MEAN_ELEMENT_THEORY', 'EPOCH', 'MEAN_MOTION', 'ECCENTRICITY', 'INCLINATION', 'RA_O

In [10]:
tle_df = pd.concat(all_records, ignore_index=True)
print('\nSample of merged DataFrame:')
print(tle_df.head())
print(f'\nTotal rows: {tle_df.shape[0]}')
print('Columns:', tle_df.columns.tolist())


Sample of merged DataFrame:
         DATE  NORAD_CAT_ID SEMIMAJOR_AXIS
0  2019-01-01         39359       6889.623
1  2019-01-02         39359       6889.619
2  2019-01-04         39359       6889.609
3  2019-01-05         39359       6889.607
4  2019-01-06         39359       6889.603

Total rows: 4810
Columns: ['DATE', 'NORAD_CAT_ID', 'SEMIMAJOR_AXIS']
